# Домашнее задание 3.
## Классификатор на сезонные ряды для построения автоматического пайплайна прогнозирования.

В рамках курса мы разбирали множество подходов к прогнозированию временных рядов, однако одновременно мы работали со счетным количеством рядов,
для каждого из которых была возможность проанализировать его "вручную" и подобрать наилучшую модель. Но что, если мы столкнулись с задачей прогнозирования
сразу сотен рядов? В таком случае мы уже не можем строить модель на каждый ряд вручную и нам придется либо строить одну модель сразу на все ряды и надеяться, что качество будет удовлетворительным, либо придумывать некий automl подход. 

Одним из вариантов такого automl подхода может служить тот, в котором мы разбиваем временные ряды на типы и для каждого типа строим модель прогнозирования, наиболее хорошо такой тип описывающую. Для того, чтобы автоматически определять тип ряда, мы строим классификатор, способный этот тип определять.

Одним из самых очевидных разбиений рядов на типы может быть разбиение на сезонные и не сезонные ряды. Предположение о наличии сезонности позволит использовать алгоритмы, лучше заточенные под поиск сезонных зависимостей, кроме того, это позволит сэкономить вычислительные ресурсы, так как несезонные ряды зачастую можно хорошо прогнозировать более легковесными методами. 

В текущем задании вам как раз и предлагается спроектировать такой бинарный классификатор, который мог бы отделять сезонные ряды от несезонных.

## Задание

Ниже приведен класс для обучения произвольного классификатора на датасете одномерных временных рядов, т.е. на наборе обьектов вида
(timeseries, label). Данный класс определяет метод для получения произвольного признакового описания ряда `get_feature_vector`, использует его для получения датасета, после чего обучает на датасете бинарный классификатор удовлетворяющий sklearn estimator API.

**Вам необходимо:**
1. Определить метод `get_feature_vector`, который позволил бы выделить из временного ряда характерные признаки, указывающие на сезонность.
2. Разбить датасет в соотношении 60/40 (train, test). Проследите за сбалансированностью классов в выборках.
3. Обучить модель на трейне.
4. Сделайте прогноз на тесте и получите метрики `f1`, `auc-roc`.
5. Итоговые баллы за задание будут зависеть от значения метрик.  
   `50 < f1,roc < 60` 4 балла  
   `60 < f1,roc < 70` 6 баллов  
   `70 < f1,roc < 85` 8 баллов  
   `85 < f1,roc < 95` 9 баллов  
   `95 < f1,roc` 10 баллов  

In [183]:
from pathlib import Path
from typing import Iterable, Tuple

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression


class SeasonalClassifier:
    def __init__(self, classifier=LogisticRegression, **kwargs):
        self.classifier = classifier(**kwargs)
        self.fitted = False

    def fit(self, ts_dataset: Iterable[Tuple[pd.Series, int]], **kwargs):
        X, y = [], []
        for ts, label in ts_dataset:
            feature_vector = self.get_feature_vector(ts)
            X.append(feature_vector)
            y.append(label)
        X = np.array(X)
        y = np.array(y)
        self.classifier.fit(X, y, **kwargs)
        self.fitted = True

    def predict(self, ts: pd.Series):
        self._check_fitted()
        feature_vector = self.get_feature_vector(ts)
        label = self.classifier.predict(np.array([feature_vector]))
        return label

    def predict_proba(self, ts: pd.Series):
        self._check_fitted()
        feature_vector = self.get_feature_vector(ts)
        proba = self.classifier.predict_proba(np.array([feature_vector]))
        return proba

    def get_feature_vector(self, ts: pd.Series):
        # your code here
        return feature_vector

    def _check_fitted(self):
        if not self.fitted:
            raise ValueError('This instance is not fitted yet. Call fit method first.')

In [167]:
# код для считывания датасета

with open('./dataset_clf/labels.csv', 'r') as f:
    labels = (line.replace('\n', '').split(',') for line in f)
    labels = dict(labels)
    labels = {k: int(v) for k, v in labels.items()}

dataset = []

for filename in Path('./dataset_clf/').glob('[!labels]*'):
    ts = pd.read_json(filename, typ='series')
    dataset.append((ts, labels[filename.name]))